In [1]:
import numpy as np
from utils.array import cyclic_padding

kid = [ cyclic_padding( np.loadtxt(f'data/feature_array/kid{i}.txt'), axis=1 )
        for i in range(18) ]
other = [ cyclic_padding( np.loadtxt(f'data/feature_array/other{i}.txt'), axis=1 )
        for i in range(16) ]

maxlen = max( img.shape[0] for img in (kid + other) )

X = np.zeros( (18+16, maxlen, 19), dtype=np.float )
# y = np.array( [[1, 0]]*18 + [[0, 1]]*16 )
y = np.array( [1]*18 + [0]* 16)

for i, img in enumerate(kid + other):
    h, _ = img.shape
    X[i, :h] = img
del kid, other

X = X[:, :, :, np.newaxis]

In [415]:
from keras.layers import Input, GlobalMaxPool2D, Conv2D, LeakyReLU, Activation
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.activations import selu
from keras.models import Model
from utils.layers import Dilated2D

inp = Input(shape=(maxlen, 19, 1) )
conv = Dilated2D(filters=1, 
                 kernel_size=(9, 10),
                 dilation_rate=((12, 7, 5, 4, 3, 3, 2, 2), 1), 
                 activation='linear',
                 kernel_regularizer=l2(0.0001),
                 use_bias=False
                 )(inp)
sig = GlobalMaxPool2D()(conv)
out = LeakyReLU(0.1)(sig)
# out = Activation('sigmoid')(out)
# true = Activation('sigmoid')(sig)
# false = 1 - true
# out = Concatenate()([true, false])
model = Model(inp, out)
model.compile(
              optimizer='adam',
            #   optimizer=SGD(lr=0.000001), 
              loss='mse'
            #   loss='binary_crossentropy'
)

In [419]:
model.fit(X, y, epochs=1000, verbose=0)
model.fit(X, y, epochs=10, verbose=1)


Epoch 1/10
34/34 [==============================] - 0s 2ms/step - loss: 3695533898812.2646
Epoch 2/10
34/34 [==============================] - 0s 2ms/step - loss: 3694217380803.8232
Epoch 3/10
34/34 [==============================] - 0s 2ms/step - loss: 3693601899580.2354
Epoch 4/10
34/34 [==============================] - 0s 2ms/step - loss: 3695617538168.4707
Epoch 5/10
34/34 [==============================] - 0s 2ms/step - loss: 3700437286912.0591
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 3703971902915.7646
Epoch 7/10
34/34 [==============================] - 0s 1ms/step - loss: 3706462527969.9409
Epoch 8/10
34/34 [==============================] - 0s 2ms/step - loss: 3708092385159.5879
Epoch 9/10
34/34 [==============================] - 0s 2ms/step - loss: 3709023520647.5591
Epoch 10/10
34/34 [==============================] - 0s 2ms/step - loss: 3709383737344.0293


In [420]:
model.predict(X)

array([[        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [-11229385.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [   116864.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [        0.],
       [    54400.],
       [        0.],
       [        0.],
       [        0.]], dtype=float32)